In [ ]:
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
#import xgboost
#from xgboost import XGBRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

In [ ]:
frames = ['df1','df2','df3','df4','df5','df6','df7','df8','df9','df10']
for i in frames:
    i=pd.DataFrame()

In [ ]:
files = ['data_CA_1','data_CA_2', 'data_CA_3', 'data_CA_4','data_TX_1','data_TX_2', 'data_TX_3','data_WI_1','data_WI_2','data_WI_3']
for i in range(10):
    frames[i] = pd.read_pickle(files[i]+'.pkl')
    print(files[i])
    print(frames[i].shape)

In [ ]:
final_frame = pd.concat(frames)

In [ ]:
final_frame.shape

In [ ]:
def pretrain_filter(dt, n):
    dt['d'] = dt['d'].apply(lambda x: int(x.split('_')[1]))
    dt['d'] = dt.d.astype('int16')
    dt = dt[dt.d > n]
    return dt

df_final = pretrain_filter(final_frame, 1200)

encoder = LabelEncoder()
df_final['store_id'] = encoder.fit_transform(df_final['store_id']).astype('int16')

df_final.info()

In [ ]:
ftrList = list(df_final.columns)
ftrList = list(set(ftrList) - set(['d', 'date', 'demand', 'store_id', 'id']))

x_train = df_final[df_final.d < 1914]
y_train = x_train['demand']

x_val = df_final[(df_final.d >= 1914) & (df_final.d < 1942)]
y_val = x_val['demand']

x_test = df_final[(df_final.d >= 1942)]

In [ ]:
def final_col(df_f):
    df = df_f[ftrList]
    col = df.columns
    col = list(col)
    print(col)
    print(type(col))
    for i in col:
        null_count = df[i].isnull().sum()
        if (null_count!=0):
            print(i,null_count)
            df[i] = df[i].fillna(0)
    return df

In [ ]:
x_train.shape

In [ ]:
x_train = final_col(x_train)

In [ ]:
x_train.shape

In [ ]:
x_val = final_col(x_val)
x_test = final_col(x_test)

In [ ]:
x_val.shape

In [ ]:
x_test.shape

## Elastic Net Model Training

In [ ]:
x_train_copy = x_train

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

In [ ]:
inner_cv = KFold(n_splits = 5, shuffle = True, random_state = 10)
outer_cv = KFold(n_splits = 5, shuffle = True, random_state = 10)
scoring='neg_root_mean_squared_error'

In [ ]:
en = ElasticNet()
en_grid = { 'alpha' : [1.0,1.5], 'l1_ratio' :[0.4], 'selection' : ['random'] } 
en_reg = GridSearchCV(estimator = en, param_grid = en_grid, scoring = scoring, cv = inner_cv)
en_m = en_reg.fit(x_train, y_train)
en_best_param = en_m.best_params_
print(en_best_param)
en_predictions = en_m.predict(x_test)
en_pred = np.array(en_predictions)
x_test_en_pred = x_test
x_test_en_pred['predictions'] = en_pred.tolist()
x_test_en_pred.to_csv('submission3-en.csv',index=False)